In [8]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import pytorch_lightning as pl

import warnings
warnings.filterwarnings('ignore')

from lazyvi import LazyVI
from data_generating_funcs import FlexDataModule, generate_linear_data
from networks import NN4vi, EarlyStopping


from utils import dropout
from sklearn import linear_model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# LazyVI Tutorial

As an example, we demonstrate the use of our LazyVI methodology on the following simple linear system:

$$f(x) = 1.5x_1 + 1.2x_2 + x_3 + \epsilon$$

Where $\epsilon \sim N(0,0.1)$ and $X \sim N(0, \Sigma_{6 \times 6})$, so the response only depends on the first three of the six variables. All variables are independent except for $x_1$ and $x_2$, whose correlation is $\rho$. Note that this is the example we use in Section 5.1 of our [paper](https://proceedings.mlr.press/v162/gao22h/gao22h.pdf).

#### Generate data
First, we generate data from this model (with $\rho$ = 0.75) using our helper functions, split it into train/test datasets, and feed it into a PyTorch compatible data module:

In [9]:
# generate data
X, y = generate_linear_data(beta=[1.5, 1.2, 1, 0, 0, 0], corr=0.75)
n, p = X.shape
dm = FlexDataModule(X,y) # feed into data module. Note that this setup automatically splits data into train/test sets

Then, we define and train the original, full neural network on all the data. We are using a network with one hidden layer that has a width of 50, instatiated using a helper function. We train the full network until convergence.

#### Train "full" network

In [10]:
width = 50
# initialize network
torch.manual_seed(711)
full_nn = NN4vi(p, [width], 1)

# train full network
early_stopping = EarlyStopping('val_loss', min_delta=1e-5)
trainer = pl.Trainer(callbacks=[early_stopping], max_epochs=1000)
trainer.fit(full_nn, dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 401   
------------------------------------
401       Trainable params
0         Non-trainable params
401       Total params
0.002     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

#### Estimate Variable Importance 

Now, we estimate the variable importance of $x_1$ using three methods: retraining, dropout, and LazyVI (our method). We show in Section 3 of our paper that the true variable importance vaue is given by $(1.5)^2(1-\rho^2) \approx 0.98$, so we can evaluate our estimates relative to this theoretical value.

We'll start by initializing and training LazyVI. The training procedure automatically cross-validates to find the optimal $\lambda$-value for the lazy adjustment.

In [12]:
# extract train/test response for measuring performance
y_train = dm.train.dataset.tensors[1]
y_test = dm.test.tensors[1]
X_test = dm.test.tensors[0]

# create a modified dataset with the first variable dropped out
j = 0
Xj = dropout(X, j)
dmj = FlexDataModule(Xj, y)
dmj.setup()
Xj_train = dmj.train.dataset.tensors[0]
Xj_test = dmj.test.tensors[0]


# setup list to store results
results = []

# LazyVI (our method)
t0 = time.time()
lv = LazyVI(full_nn, lambda_path=np.logspace(1,3,20)) # initialize module with fully trained network, define regularization path
lv.fit(Xj_train, y_train) # fit LazyVI
tlazy = time.time() - t0

Next, we extract the VI and associated standard error and an estimate quite close to the theoretical value (and certainly within the confidence bounds):

In [13]:
lazy_vi, lazy_se = lv.calculate_vi(X_test, Xj_test, y_test, se=True)
results.append(['Lazy', tlazy, lazy_vi])
print('LazyVI Estimate:', round(lazy_vi, 4))
print('95% CI:', (round(lazy_vi - 1.96*lazy_se, 4), round(lazy_vi + 1.96*lazy_se, 4)))

LazyVI Estimate: 0.9922
95% CI: (0.923, 1.0614)


We compare these estimates to the dropout and retraining methods outlined in our paper:

In [15]:
# dropout VI
t0 = time.time()
dr = full_nn(Xj_test)
dr_vi = nn.MSELoss()(y_test, dr).item()-nn.MSELoss()(y_test, full_nn(X_test)).item()
results.append(['Dropout', time.time() - t0, dr_vi])

# retrain VI
t0 = time.time()
red_nn = NN4vi(p, [width], 1)
early_stopping = EarlyStopping('val_loss', min_delta=1e-5)
trainer = pl.Trainer(callbacks=[early_stopping], max_epochs=100)
trainer.fit(red_nn, dmj)
rt = red_nn(Xj_test)
rt_vi = nn.MSELoss()(y_test, rt).item() - nn.MSELoss()(y_test, full_nn(X_test)).item()
results.append(['Retrain', time.time() - t0, rt_vi])

# Look at results
df = pd.DataFrame(results, columns=['Method', 'Time', 'MSE'])
df

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 401   
------------------------------------
401       Trainable params
0         Non-trainable params
401       Total params
0.002     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

,Method,Time,MSE
0,Lazy,1.881637,0.992200
1,Dropout,0.001211,2.313760
2,Retrain,8.372328,0.993968


We see that, while Dropout is very fast, the VI estimate is highly inaccurate, while the Retrain and LazyVI estimates are very close to one another but LazyVI is significantly faster.